In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData
from concurrent.futures import ThreadPoolExecutor, as_completed
pd.set_option('display.max_rows', 20)
pd.options.mode.copy_on_write = True

In [2]:
#Base URL for API call
base = "https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes"
crashesQuery = "/GetCrashesByVehicle"
detailsQuery = "/GetCaseDetails"
threads = 50
isLocal = True
maxStates = 50
minYear = 2018
maxYear = 2021
params = {
   # 'fromYear': 2018,
   # 'toYear': 2019,
    'format': 'json',
    'minNumOfVehicles': 1,
    #'states': 1,  
    'maxNumOfVehicles': 100
}

In [6]:
#Construct a dataframe looping through different states and years to retrieve info related to each case

def getAllCases():
    if isLocal:
        return readLocalDb()
    else:
        allCases = []   
        for state in range(1, maxStates):
            for year in range(minYear, maxYear):
                params['fromYear'] = year
                params['toYear'] = year
                params['states'] = state
                response = requests.get(base + crashesQuery, params=params).json()
                allCases += response["Results"][0]
        df = pd.DataFrame(allCases)
        persist(df)
        return df

#List to hold crash case details
def findCaseDetails(cases, min, max):
    batch = []
    print("Looking up (" + str(min) + ", " + str(max) + ")")

# Data to be pulled from JSON API and added to batch list
    for idx in range(min, max):
        caseRow = cases.iloc[idx,:]
        print("Batch#" + str(idx))
        caseDetail = requests.get(base + detailsQuery, params = {
            'stateCase': caseRow["St_Case"],
            'caseYear': caseRow["CrashDate"].year,
            'state': caseRow["State"],
            'format': 'json'
        }).json()["Results"][0][0]['CrashResultSet']
        vehicles = getVehiclesMakeList(caseDetail)
        d = {
            "ST_CASE": caseDetail["ST_CASE"],
            "STATENAME": caseDetail["STATENAME"],
            "WEATHER": caseDetail["WEATHER"],
            "WEATHER1": caseDetail["WEATHER1"],
            "WEATHER2": caseDetail["WEATHER2"],
            "FATALS": caseDetail["FATALS"],
            "YEAR": caseDetail["YEAR"],
            "YEAR": caseDetail["YEAR"],
            "VEHICLES": ','.join(vehicles),
            "IS_DUI": isDUIRelated(caseDetail),
        }
        batch.append(d)
    return batch

def getVehiclesMakeList(detail):
    make_name = [item['MAKENAME'] for item in detail['Vehicles']]
    return make_name

def isDUIRelated(detail):
    return detail['DRUNK_DR'] == '1'

def seat_position(detail):
    seat_position =[item['SEAT_POSNAME'] for item in detail['Person']]
    return seat_position

# Clean databases in order to persist DataFrame contents into databases   
def cleanDb(engine):
    metadata = MetaData()
    metadata.reflect(bind=engine)
    metadata.drop_all(engine)
    
def persist(df):
    engine = create_engine('sqlite:///crashes.db')
    cleanDb(engine)
    df.to_sql('crashes', con=engine, if_exists='replace', index=False)
    engine.dispose()

def persistDetails(df):
    engine = create_engine('sqlite:///details.db')
   # cleanDb(engine)
    df.to_sql('details', con=engine, index=True)
    engine.dispose()

def readLocalDb():
    engine = create_engine('sqlite:///crashes.db')
    df = pd.read_sql_table('crashes', con=engine)
    engine.dispose()
    return df

# Rerieve data stored in "details" of database and load it into a pandas DataFrame
def readLocalDbDetails():
    engine = create_engine('sqlite:///details.db')
    df = pd.read_sql_table('details', con=engine)
    engine.dispose()
    return df

# Process cases in parallel using concurrent threads and store the required information
def loadDetailsInParallel(cases):
    max_cases = len(cases)
    step = max_cases // threads
    windows = [(i*step, (i+1)*step) for i in range(threads)]
    windows[-1] = (windows[-1][0], max_cases)
    details = []
    with ThreadPoolExecutor(max_workers=threads) as executor:
        future_to_url = {executor.submit(findCaseDetails, cases, min, max) for min, max in windows}
        for future in as_completed(future_to_url):
            data = future.result()
            details.extend(data)
            
    df = pd.DataFrame(details)
    persistDetails(df)
    return df

In [4]:
allCasesRawFrame = getAllCases()

# filter out timestamps that are problematic
allCases = allCasesRawFrame[allCasesRawFrame['CrashDate'].str.count('-') == 1]

allCases.loc[:,'CrashDate'] = allCases['CrashDate'].apply(lambda x: datetime.fromtimestamp(int(x[6:19])/1000))
allCases


,CountyName,CrashDate,Fatals,Peds,Persons,St_Case,State,StateName,TotalVehicles
0,COFFEE (31),2017-04-02 18:23:00,1,0,1,10206,1,Alabama,1
1,LEE (81),2017-04-05 23:25:00,1,0,2,10207,1,Alabama,1
2,PIKE (109),2017-04-07 20:55:00,1,1,1,10208,1,Alabama,1
3,LOWNDES (85),2017-04-10 10:53:00,1,0,1,10209,1,Alabama,1
4,DALE (45),2017-04-07 10:00:00,2,0,8,10210,1,Alabama,4
...,...,...,...,...,...,...,...,...,...
167218,SALT LAKE (35),2021-09-04 02:44:00,1,0,3,490307,49,Utah,1
167219,SALT LAKE (35),2021-11-06 00:38:00,1,0,2,490308,49,Utah,1
167220,WASHINGTON (53),2021-12-17 14:48:00,1,0,5,490309,49,Utah,2
167221,UTAH (49),2021-09-14 11:45:00,1,0,5,490310,49,Utah,4


In [5]:
allCases['CrashDate'] = pd.to_datetime(allCases['CrashDate'])
allCases
allCases.dtypes

CountyName               object
CrashDate        datetime64[ns]
Fatals                    int64
Peds                      int64
Persons                   int64
St_Case                   int64
State                     int64
StateName                object
TotalVehicles             int64
dtype: object

In [6]:


# Ignore time of day, so we could group by day
allCases.loc[:, 'CrashDate'] = allCases['CrashDate'].dt.normalize()


#details = loadDetailsInParallel(allCases)
#readLocalDbDetails()
# for idx, row in enrichedCases.iterrows():
#     print("Loading Details...")
#     caseDetail = findCaseDetails(row)
#     vehicles = getVehiclesMakeList(caseDetail)
#     enrichedCases.at[idx, 'CarMakeList'] = cleanVehicleMakes(vehicles)
#     enrichedCases.at[idx, 'isDUI'] = isDUIRelated(caseDetail)
#     enrichedCases.at[idx, 'WEATHER'] = caseDetail['WEATHER']
#     enrichedCases.at[idx, 'WEATHER1'] = caseDetail['WEATHER1']
#     enrichedCases.at[idx, 'WEATHER2'] = caseDetail['WEATHER2']
#     enrichedCases.at[idx, 'FATALS'] = caseDetail['FATALS']
#print("Saving")
#enrichedCases

#print("Done Saving")
#readLocalDb()
    
